In [1]:
dbutils.fs.put("dbfs:/splunkUF/splunkUF-init.sh" ,"""
#https://www.splunk.com/en_us/download/universal-forwarder.html
ufDlUrl="https://download.splunk.com/products/universalforwarder/releases/8.0.2.1/linux/splunkforwarder-8.0.2.1-f002026bad55-Linux-x86_64.tgz"
ufDlDir="/dbfs/splunkUF"
ufInstallDir="/local_disk0"
TARGETURI="<FQDN of Splunk Deployment Server>:8089"

ufDlFilename=$(basename "${ufDlUrl}")
ufDlPath="${ufDlDir}/${ufDlFilename}"
SPLUNK_HOME="${ufInstallDir}/splunkforwarder"
SPLUNK_ARGS="--accept-license --answer-yes --no-prompt"

# If Splunk is installed, start splunk and exit
if [ -e ${SPLUNK_HOME} ]; then
  echo "#### ${SPLUNK_HOME} exists! Starting Splunk. ####"
  ${SPLUNK_HOME}/bin/splunk start ${SPLUNK_ARGS}
  exit 0
fi

# If the splunk unervisal forwarder installer does not exist, download it from the splunk website
if ! [ -e ${ufDlPath} ]; then
  if [ "${DB_IS_DRIVER}" = "TRUE" ]; then
    if ! [ -e ${ufDlDir} ]; then mkdir ${ufDlDir}; fi
    echo "#### Downloading ${ufDlFilename} to ${ufDlDir} ####"
    curl -s "${ufDlUrl}" -o ${ufDlPath}
  else
    echo "#### Sleeping while driver downloads ${ufDlFilename} to ${ufDlDir} ####"
    i=0; while ! [ -e ${ufDlPath} ] && [ $i -lt 30 ]; do let i=i+1; printf "."; sleep 1; done
    echo
  fi
fi

# exit if the splunk installer file does not exist
if ! [ -e ${ufDlPath} ]; then
  echo "#### ${ufDlPath} not found! Exiting.####"
  exit 1
fi

# Install and configure splunk
echo "#### Installing from ${ufDlPath} ####"
if ! [ -e "${ufInstallDir}" ]; then mkdir "${ufInstallDir}"; fi
tar -xzf ${ufDlPath} -C "${ufInstallDir}"

#Configure deployment client
mkdir -p ${SPLUNK_HOME}/etc/apps/local/default
cat > ${SPLUNK_HOME}/etc/apps/local/default/deploymentclient.conf << EOF
[deployment-client]
clientName = dbr-${HOSTNAME}
[target-broker:deploymentServer]
targetUri = ${TARGETURI}
EOF

#Add DB_CLUSTER_ID & DB_IS_DRIVER as indexed fields/meta data to the events
mkdir -p ${SPLUNK_HOME}/etc/apps/local/default
cat > ${SPLUNK_HOME}/etc/apps/local/default/inputs.conf << EOF
[default]
_meta = DB_CLUSTER_ID::${DB_CLUSTER_ID} DB_IS_DRIVER::${DB_IS_DRIVER}
EOF

${SPLUNK_HOME}/bin/splunk start ${SPLUNK_ARGS}
""", True)

#Splunk Indexing Tier
###create index
```
mkdir -p ${SPLUNK_HOME}/etc/apps/dbr/local

cat > ${SPLUNK_HOME}/etc/apps/dbr/local/indexes.conf << EOF
[cluster_logs]
coldPath = \$SPLUNK_DB/\$_index_name/colddb
homePath = \$SPLUNK_DB/\$_index_name/db
maxTotalDataSizeMB = 4096
thawedPath = \$SPLUNK_DB/\$_index_name/thaweddb
EOF
```

###temporarily disable TRUNCATE
```
cat > ${SPLUNK_HOME}/etc/apps/dbr/local/props.conf << EOF
[default]
TRUNCATE = 0
EOF
```

###create inputs
```
cat > ${SPLUNK_HOME}/etc/apps/dbr/local/inputs.conf << EOF
[splunktcp://9997]
connection_host = ip
EOF
```

#Splunk UF Configs (served via Deployment Server)

###create serverclass
```
mkdir -p ${SPLUNK_HOME}/etc/apps/dbr/local
cat > ${SPLUNK_HOME}/etc/apps/dbr/local/serverclass.conf << EOF
[serverClass:dbr_hosts:app:dbr]
restartSplunkWeb = 0
restartSplunkd = 1
stateOnClient = enabled

[serverClass:dbr_hosts]
machineTypesFilter = linux-x86_64
whitelist.0 = dbr-*
EOF
```

###create inputs
```
mkdir -p ${SPLUNK_HOME}/etc/deployment-apps/dbr/default

cat > ${SPLUNK_HOME}/etc/deployment-apps/dbr/default/inputs.conf << EOF
[monitor:///databricks/driver/logs]
disabled = false
index = cluster_logs

[monitor:///databricks/spark/work]
disabled = false
index = cluster_logs

[monitor:///databricks/spark/logs]
disabled = false
index = cluster_logs

[monitor:///databricks/driver/eventlogs]
disabled = false
index = cluster_logs

[monitor:///databricks/driver/derby.log]
disabled = false
sourcetype = derby
index = cluster_logs

[monitor:///databricks/init_scripts/*.log]
disabled = false
sourcetype = init_scripts
index = cluster_logs

[monitor:///databricks/chauffeur/logs]
disabled = false
index = cluster_logs

[monitor:///databricks/data/logs]
disabled = false
index = cluster_logs
EOF
```

###create outputs
```
cat > ${SPLUNK_HOME}/etc/deployment-apps/dbr/default/outputs.conf << EOF
[tcpout]
defaultGroup = default-autolb-group

[tcpout:default-autolb-group]
server = <FQDN of Splunk Indexer>:9997

[tcpout-server://<FQDN of Splunk Indexer>:9997]
EOF
```